In [1]:
import xarray as xr
import iris
import numpy as np 
import pylab as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [88]:

### Regional gridded output from a Regional Climate Model (RCM)
hi_t2   = xr.open_dataset('data/hifid_t2m_monthly.nc')
hi_lats = hi_t2['latitude'].values
hi_lons = hi_t2['longitude'].values 

### Global gridded reanalysis (data assimilated) used to update/force the 
### lateral boundaries of the RCM
lo_t2  = xr.open_dataset('data/lofid_t2m_monthly.nc')
lo_u10 = xr.open_dataset('data/lofid_u10_monthly.nc')
lo_v10 = xr.open_dataset('data/lofid_v10_monthly.nc')

### work with numpy arrays
hi_t2_arr  = hi_t2['t2m'].values
lo_t2_arr  = lo_t2['t2m'].values 
lo_u10_arr = lo_u10['u10'].values
lo_v10_arr = lo_v10['v10'].values
time_dim   = lo_t2['time'].values 

##low fidelity lat and lon 
lo_lats = lo_t2['latitude'].values
lo_lons = lo_t2['longitude'].values

# high fidelity elevation data 
hi_elev = xr.open_dataset('data/hifid_hgt.nc')['hgt'].values



In [41]:
def NN_1(x, y, x_low):
    # Reshape the input elevation and high fidelity temperature data
    #create a new array with and extra time dimension with
    #y is elevation hi fidelity data 
    #x is temperature hi fidelity data 
    
    #new_y = np.zeros((x.shape[0],y.shape[0],y.shape[1]))
    
    new_y = x.copy()
    new_y[:,:,:] = 0
    
    new_y = np.array([y]*x.shape[0])
    #for i in range(x.shape[0]):
    #    new_y[0,:,:] = x[:,:]
    
    x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    
    # Define the model architecture
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(x.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(new_y.shape[1] * new_y.shape[2], activation='linear')) # output layer
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train the model
    model.fit(x, y, epochs=10)
    
    # Use the model to generate high-resolution temperature data
    elev_low_predict = model.predict(x_low)
    
    # Reshape the high-resolution data to the desired shape
    #high_res_data = high_res_data.reshape(high_res_shape)
    
    return elev_low_predict


In [84]:
def train_temp_model(temp_data, elevation_data, temp_data_lowres):
    # Convert data to float32
    temp_data = temp_data.astype(np.float32)
    elevation_data = elevation_data.astype(np.float32)

    # Define the model
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(temp_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(elevation_data.shape[1]))
    

    # Compile the model
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(optimizer='adam', loss='mean_squared_error')
    # Train the model
    model.fit(temp_data, elevation_data, epochs=100, batch_size=32, verbose = False) # verbose = 0 showing no training progress
    elev_pred = model.predict(temp_data_lowres)
    #model.evaluate(temp_data,elevation_data)
    #print ('accuracy = ', accuracy)
    return elev_pred


In [87]:
print(hi_t2_arr.shape)
print(lo_t2_arr.shape)

(468, 93, 87)
(468, 54, 52)


In [86]:
#for i in range(h1_t2_arr.shape[0]):
#for i in range(2):
model = train_temp_model(hi_t2_arr[0,:,:], hi_elev, lo_t2_arr[0,:,:])
#model.evaluate(hi_t2_arr[0,:,:], hi_elev)
#model.predict(lo_t2_arr[])


ValueError: Error when checking input: expected dense_42_input to have shape (87,) but got array with shape (52,)

In [76]:
a = model.predict(hi_t2_arr[0,:,:])
a.shape
hi_t2_arr.shape

(468, 93, 87)

In [42]:

lo_elev = NN_1(hi_t2_arr,hi_elev,lo_t2_arr)
'''
print(lo_t2_arr.shape)
x = lo_t2_arr.reshape(lo_t2_arr.shape[0],lo_t2_arr.shape[1]*lo_t2_arr.shape[2])
x.shape
'''

2023-02-01 12:15:34.125393: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 12:15:34.127191: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


ValueError: Input arrays should have the same number of samples as target arrays. Found 468 input samples and 93 target samples.

In [16]:
hi_elev

<xarray.Dataset>
Dimensions:    (latitude: 93, longitude: 87)
Coordinates:
  * latitude   (latitude) float32 -10.82 -10.79 -10.75 ... -7.569 -7.532 -7.496
  * longitude  (longitude) float32 280.5 280.6 280.6 280.6 ... 283.6 283.6 283.7
Data variables:
    hgt        (latitude, longitude) float32 ...
Attributes:
    Description:  High Fidelity Climate Model Output